# Train MLP Neural Network. 

TODO: set up git repository (in vscode)

In [166]:
from collections import Counter
import numpy as np
import functools
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import warnings

warnings.filterwarnings("ignore")

In [167]:
# Data loader for validation. 
class PickDataset(Dataset):
    def __init__(self, pools, packs, pick_vectors, cardnames):
        # Input is numpy arrays
        self.pools = pools
        self.packs = packs
        self.pick_vectors = pick_vectors
        self.cardnames = cardnames
                            
    def __len__(self):
        return len(self.packs)

    def __getitem__(self,index):
        return torch.from_numpy(self.pools[index]), torch.from_numpy(self.packs[index]), torch.from_numpy(self.pick_vectors[index])

In [168]:
# Load datasets. 
dataset_folder = "/Users/danielbrooks/Desktop/Code/statistical-drafting/datasets/BLB/"

train_dataset = torch.load(dataset_folder + "pick_train_dataset_test.pth")
train_dataloader = DataLoader(train_dataset, batch_size=2500, shuffle=True)

val_dataset = torch.load(dataset_folder + "pick_val_dataset_test.pth") 
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [169]:
# class Siamese(nn.Module):
#     def __init__(self, input_size, output_dim):
#         super(Siamese,self).__init__()
#         self.input_size = input_size
        
#         self.hidden_1 = nn.Sequential(
#             nn.Linear(input_size, 400),
#             # nn.BatchNorm1d(400),  # Add BatchNorm after Linear
#             nn.Dropout(0.5),
#             nn.ELU()
#         )
# #         self.hidden_2 = nn.Sequential(
# #             nn.Linear(64, 32),
# #             nn.Dropout(0.5),
# #             nn.ELU()
# #         )
#         self.out = nn.Sequential(
#             nn.Linear(400, output_dim),
#             nn.Tanh()
#         )

#     def forward(self,x):
#         x = self.hidden_1(x)
#         # x = self.hidden_2(x)
#         x = self.out(x)
#         return x

# def get_distance(positive,negative):
#     return torch.sum(torch.pow(positive-negative,2),dim=1)

In [170]:
def get_card_embeddings(network, set_size=271):
    """
    Returns card embeddings from the network. 
    
    Can be cached after network training. 
    """
    network.eval()
    with torch.no_grad(): 
        card_embeddings = network(torch.eye(set_size))
    return card_embeddings

def make_pick(network, pool, pack, card_embeddings):
    """
    Makes a pick from a pack given a trained network. 
    
    pool: Tensor (1 x set_size)
    pack: Tensor (1 x set_size)
    card_embeddings: (set_size x output_dim)
    network: torch.nn
    
    returns: Tensor (1 x set_size) - recommended pick
    """
    # Make sure no gradients are saved
    network.eval()
    with torch.no_grad():
        
        # Get embedding of card pool. 
        pick_embedding = network(pool.float())
    
    # Compare to pack
    cards_in_pack = torch.nonzero(pack)[:, 1]
    pack_card_embeddings = card_embeddings[cards_in_pack, :]
    pack_distances = get_distance(pack_card_embeddings, pick_embedding)

    # Make pick
    pick_distance, pack_index = torch.min(pack_distances, dim=0) # torch type. 
    pick_index = cards_in_pack[pack_index.item()].item()
    
    # Return number and one hot vector
    pick_vector = torch.zeros([1, 271], dtype=torch.int64)
    pick_vector[0, pick_index] = 1
    return pick_index, pick_vector

def evaluate_model(val_dataloader, network):
    # Load data.
    t0 = time.time()
    
    # Refresh card embeddings. 
    card_embeddings = get_card_embeddings(network)

    # Count number correct picks. 
    num_correct, num_incorrect = 0, 0
    for pool, pack, human_pick in val_dataloader: # Assumes batch size of 1.      
        bot_pick_index, bot_pick_vector = make_pick(network, pool, pack, card_embeddings)

        # Print results
        if torch.equal(human_pick, bot_pick_vector):
            num_correct += 1
        else:
            num_incorrect += 1

    # Untrained, random picks. 
    # print(f"total picks = {num_correct + num_incorrect}")
    print(f"correct = {round(100 * num_correct / (num_correct + num_incorrect), 1)}%")
    # print(f"evaluation runtime = {round(time.time() - t0, 2)}s")

In [171]:
# def train_siamese(train_dataloader, val_dataloader, epochs, network, experiment_name):    
#     loss_fn = torch.nn.TripletMarginLoss()
#     # optimizer = optim.Adam(network.parameters(), lr = 1e-4)
#     optimizer = optim.Adam(network.parameters(), lr = 0.001) # try this with batchnorm. 

#     # Initial evaluation. 
#     print("Starting to train model")
#     evaluate_model(val_dataloader, network)
    
#     num_picks = 0
    
#     t0 = time.time()    
#     # Training loop. 
#     for epoch in range(epochs):
#         network.train()
#         epoch_training_loss = list()

#         print(f"\nStarting epoch {epoch}")
        
#         # Train model. 
#         for i, (anchor, positive, negative) in enumerate(train_dataloader):
#             optimizer.zero_grad()
#             out1 = network(anchor.float()) # Cast types as float to facilitate running. 
#             out2 = network(positive.float())
#             out3 = network(negative.float())
#             loss = loss_fn(out1, out2, out3)
#             loss.backward()
#             optimizer.step()
#             epoch_training_loss.append(loss.item())
            
#             # Note 1.6 million picks in training set. 
#             examples_processed = (i + 1) * anchor.shape[0]
#             if examples_processed % 250000 == 0:
#                 print(f"training complete on {examples_processed} examples, time={round(time.time() - t0, 1)}")
            
#         # Evaluation. 
#         network = network.eval()
#         print(f"Training loss: {round(np.mean(epoch_training_loss), 4)}")
#         evaluate_model(val_dataloader, network)
        
#         # Save network weights
#         weights_path = f'./model_weights/{experiment_name}.pt'
#         print(f"Saving model weights to {weights_path}")
#         torch.save(network.state_dict(), weights_path)
            
#     return network

In [172]:
# TODO: build training infra (model serialization, metric logging (mlflow?), using model as a pick recommender)

In [173]:
# # Sample evaluation flow - verified that this works. 
# # network = Siamese(271, 400)

# EXPERIMENT_NAME = "exp27_5weeks_400_400"
# network = train_siamese(train_dataloader,
#                         val_dataloader,
#                         epochs=2000,
#                         network=network,
#                         experiment_name=EXPERIMENT_NAME)

In [174]:
# Experiment #2 (d=256 output layer) -> appeared to overfit during first 5 epochs. Reduce output dimension to 32
# Experiment #3 (d=32 output layer) -> convered to 67.5% test accuracy after 65 epochs
# Experiment #4 (d=32 output, include plat, ~x5 dataset size, currently at 66.8% after 29 epochs) (no network)
# Experiment #5 (d=256 output, include plat, ~x5 dataset size, at at 57.4% after 23 epochs) (results in exp 4, oops)
# Experiment #6 (d=64 output, include plat, ~x5 dataset size, 67.2% on epoch 18) "exp6_platinum_64outputneuron"
# Experiment #7 (d=64 output, only last 3 days w/ plat, 66.4% after 25 epochs (train loss ~0.2587)
#               (66.9%, loss down to 0.2388 with LR=0.001))
#               (extending with dropout off, loss to 0.2300)
#               (Finish with accuracy of 66.9%, loss of 0.2252)
# Experiment #8 (128, 64, 64) no dropout -> 67.4% / 0.2242 in just 10 epochs
# Experiment #9 (64, 32, 32) -> 67.3% / 0.22 (Eventually train loss = 0.2014, correct% = 66.2)
# Experiment #10 (128, 128) -> 67.4% / 0.2400 (in 9 epochs), peaked at 67.5%
# Experiment #11 (16, 16) - seems to underfit (correct 60.5, loss = 0.4028)
# Experiment #13 (4096, 4096) - a bit slow (10 minutes/epoch), 66.0%/0.2245 after epoch 2, 66.9 after a few
# Experiment #14 (64, 16) Training loss: 0.2843 correct = 63.7%  ?? not sure on dimension
# Experiment #15 (64, 16) confirmed 14 (63.7%)
# Experiment #16 (16, 64) slower convergence (~62.1% early on)
# Experiment #17 (16, 16) Training loss: 0.4143 correct = 58.6%
# Experiment #18 (64, 64) epoch 12 Training loss: 0.2608 correct = 66.3%
# Experiment #19 (276, 276) Training loss: 0.2259, correct = 67.6% (epoch 7), LR = 0.003 (58s/epoch)
             # Continue with batch_size=10k (speeds up training by ~20%?), correct -> 67.8%
             # Trying batch_size = 100k now (no speed up now), similar perf
             # Trying without dropout
# Experiment #20 (512, 512) LR 0.01 initially, Training loss: 0.2014, correct = 68.2% (peak)
# Experiment #21 (1024, 1024) LR 0.1 initially (3 minutes/epoch)
            # Training loss: 0.2287, correct = 66.6%
# Experiment #22 (400, 400) 68.3% peak, (~40 epochs), then overfit

# MOST RELIABLE
# Experiment #23 (400, 400) small batch (25) - good & steady perf -  (~6 mins/epic) Training loss: 0.216, correct = 68.3% after eopch 20
# Experiment #24 (400, 400) batchnorm, no dropout (lr=0.01, batchsize=2500), (time_per_epoch=~1.3 minutes)
            # Network overfits immediately with batchnorm... (time per epoch no batchnorm 72 seconds)
            # No batchnorm, no dropout: Training loss: 0.2295, correct = 66.8% (after epoch 1)
            #                           Similar perf after epoch 6 (maybe need dropout?)

# FASTEST CONVERGENCE TO ~1% OF OPTIMAL            
# Experiment #25 (400, 400) dropout 0.2, batchsize=2500, learningrato=0.01
            # Time per epic ~80 seconds
            # Training loss: 0.2305, correct = 67.0% (after epic 1 - 2 minutes of training)
            # 67.3 after 15 epoch, switch to LR = 0.001
            # 67.7 after another 10 epochs with low LR (Training loss: 0.1869)
            # Increase dropout to 0.4 ...
            # Now trying dropout=0.5, LR=0.001 (best)
            # After 16 epoch: Training loss: 0.2193 correct = 68.1%

# Experiment #26 (400, 400) Try MLP Model
# Experiment #27 full 5 weeks of data (400, 400) 

## Train MLP (experiment)

In [175]:
# class MLP(nn.Module):
#     def __init__(self, input_size):
#         super(MLP,self).__init__()
#         self.input_size = input_size
  
#         self.MODEL_WIDTH = 100
        
#         self.hidden_1 = nn.Sequential(
#             nn.Linear(input_size, self.MODEL_WIDTH),
#             nn.BatchNorm1d(self.MODEL_WIDTH),  # Add BatchNorm after Linear
#             nn.Dropout(0.5),
#             nn.ELU()
#         )
#         self.hidden_2 = nn.Sequential(
#             nn.Linear(self.MODEL_WIDTH, self.MODEL_WIDTH),
#             nn.BatchNorm1d(self.MODEL_WIDTH),  # Add BatchNorm after Linear
#             nn.Dropout(0.5),
#             nn.ELU()
#         )
#         self.out = nn.Sequential(
#             nn.Linear(self.MODEL_WIDTH, input_size),
# #             nn.Tanh()
#         )

#     def forward(self,x, pack):
#         x = self.hidden_1(x)
# #         x = self.hidden_2(x)
#         x = self.out(x)
#   #       print("before pack", x.shape)
#         x = x * pack
#   #      print("after pack", x.shape)
#         return x

In [176]:
class ModernMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout=0.1):
        """
        Args:
            input_dim (int): Size of the input features.
            hidden_dims (list): List of integers specifying hidden layer sizes.
            output_dim (int): Size of the output features.
            dropout (float): Dropout rate for regularization.
        """
        super(ModernMLP, self).__init__()
        
        # Input layer
        self.input_layer = nn.Linear(input_dim, hidden_dims[0])
        
        # Hidden layers
        self.hidden_layers = nn.ModuleList(
            nn.Linear(hidden_dims[i], hidden_dims[i+1]) for i in range(len(hidden_dims) - 1)
        )
        
        # Projection layers for residuals if dimensions mismatch
        self.projections = nn.ModuleList(
            nn.Linear(hidden_dims[i], hidden_dims[i+1]) if hidden_dims[i] != hidden_dims[i+1] else nn.Identity()
            for i in range(len(hidden_dims) - 1)
        )
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)
        
        # Normalization and regularization
        self.norms = nn.ModuleList(nn.LayerNorm(dim) for dim in hidden_dims)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, pack):
        # Input layer
        x = self.input_layer(x)
        x = F.gelu(x)  # Activation function
        x = self.dropout(x)
        
        # Hidden layers with residual connections
        for layer, norm, proj in zip(self.hidden_layers, self.norms, self.projections):
            residual = x
            x = layer(x)
            x = F.gelu(x)
            x = self.dropout(x)
            x = norm(x + proj(residual))  # Residual connection + LayerNorm
            
        # Output layer
        x = self.output_layer(x)
        x = x * pack
        return x

In [177]:
def evaluate_mlp(val_dataloader, mlp_network):
    # Load data.
    t0 = time.time()

    # Count number correct picks. 
    num_correct, num_incorrect = 0, 0
    for pool, pack, human_pick_vector in val_dataloader: # Assumes batch size of 1. 
        
        human_pick_index = torch.argmax(human_pick_vector.int(), 1)
        mlp_network.eval()
        with torch.no_grad():
            bot_pick_vector = mlp_network(pool.float(), pack.float())
            bot_picks_index = torch.argmax(bot_pick_vector, 1)

        # print(human_pick_vector, bot_pick_vector)
        # print(human_pick_index, bot_picks_index, torch.equal(human_pick_index, bot_picks_index))
        
        # Print results
        if torch.equal(human_pick_index, bot_picks_index):
            num_correct += 1
        else:
            num_incorrect += 1

    # Untrained, random picks. 
    # print(f"total picks = {num_correct + num_incorrect}")
    correct_percent = 100 * num_correct / (num_correct + num_incorrect)
    print(f"correct = {round(correct_percent, 1)}%")
    return correct_percent

In [178]:
def train_mlp(train_dataloader, val_dataloader, epochs, network, experiment_name):    
    # loss_fn = torch.nn.TripletMarginLoss()
    loss_fn = torch.nn.CrossEntropyLoss()
    # optimizer = optim.Adam(network.parameters(), lr = 1e-4)
    optimizer = optim.Adam(network.parameters(), lr = 0.01) # try this with batchnorm. 

    # Initial evaluation. 
    print("Starting to train MLP model")
    best_percent_correct = evaluate_mlp(val_dataloader, network)
    # evaluate_model(val_dataloader, network)
    
    num_picks = 0
    t0 = time.time()    
    # Training loop. 
    for epoch in range(epochs):
        network.train()
        epoch_training_loss = list()

        print(f"\nStarting epoch {epoch}")
        # Train model. 
        for i, (pool, pack, pick_vector) in enumerate(train_dataloader):
            optimizer.zero_grad()
            predicted_pick = network(pool.float(), pack.float())
            loss = loss_fn(predicted_pick, pick_vector.float())
            loss.backward()
            optimizer.step()
            epoch_training_loss.append(loss.item())
            
            # Note 1.6 million picks in training set. 
            examples_processed = (i + 1) * pool.shape[0]
            if examples_processed % 100000 == 0:
                print(f"training complete on {examples_processed} examples, time={round(time.time() - t0, 1)}")
        
        print(f"Training loss: {round(np.mean(epoch_training_loss), 4)}")

        
        if epoch % 3 == 0 and epoch > 0:
            # Evaluation. 
            network = network.eval()
            percent_correct = evaluate_mlp(val_dataloader, network)

            # Save best model. 
            if percent_correct > best_percent_correct:
                best_percent_correct = percent_correct
                weights_path = f'./model_weights/{experiment_name}.pt'
                print(f"Saving model weights to {weights_path}")
                torch.save(network.state_dict(), weights_path)                
            
    return network

In [179]:
# Load datasets. 
dataset_folder = "/Users/danielbrooks/Desktop/Code/statistical-drafting/datasets/BLB/"

mlp_train_dataset = torch.load(dataset_folder + "pick_train_dataset_test.pth")
mlp_train_dataloader = DataLoader(train_dataset, batch_size=10000, shuffle=True)

val_dataset = torch.load(dataset_folder + "pick_val_dataset_test.pth") 
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [180]:
# Sample evaluation flow - verified that this works. 
mlp_network = ModernMLP(input_dim=271, hidden_dims=[400, 400], output_dim=271, dropout=0.5)
EXPERIMENT_NAME = "exp33_ModernMLP_nobasics_diamond_weeks25_400_400"
network = train_mlp(mlp_train_dataloader,
                        val_dataloader,
                        epochs=2000,
                        network=mlp_network,
                        experiment_name=EXPERIMENT_NAME)

# Exp 26 - result 66.7% for simple MLP 2 layer - 500, large batch size helps. 
# Exp 27 - result 65.6% for various experiments. 
# Exp 28 - modern MLP architecture from chatgpt
           # 20 epoch 67.7% - 0.91 
           # 25 epoch 67.9% - 0.8982
           # 30 epoch 68.0% -  0.882
           # 35 epoch 67.9% - 0.8627
           # ... overfit
# Exp 29 - low batch size & learning rate, dropout=0.5
           # Stable performance, peaking at 67.6%
           # peaks at ~67.9 for 5 hidden layer
# Exp 30 - large batch size / learning rate 
# Exp 31 - full dataset - 66.8%, converges very quickly
# Exp 32 - exp32_ModernMLP_diamond_weeks25_400_400 70.8%
# Exp 33 - exp33_ModernMLP_nobasics_diamond_weeks25_400_400 70.8%

Starting to train MLP model
correct = 19.4%

Starting epoch 0
training complete on 100000 examples, time=3.2
training complete on 200000 examples, time=4.5
training complete on 300000 examples, time=5.8
training complete on 400000 examples, time=7.9
training complete on 500000 examples, time=9.3
training complete on 600000 examples, time=10.7
training complete on 700000 examples, time=12.3
Training loss: 1.0562

Starting epoch 1
training complete on 100000 examples, time=15.5
training complete on 200000 examples, time=16.8
training complete on 300000 examples, time=19.0
training complete on 400000 examples, time=20.6
training complete on 500000 examples, time=21.9
training complete on 600000 examples, time=23.1
training complete on 700000 examples, time=25.2
Training loss: 0.8121

Starting epoch 2
training complete on 100000 examples, time=27.6
training complete on 200000 examples, time=29.0
training complete on 300000 examples, time=31.1
training complete on 400000 examples, time=32.4

KeyboardInterrupt: 

In [181]:
# Conclusion - can actually build with MLP network. 

In [182]:
# Things to do:

# Dataloader:
# 1. Download data from 5 week chunk (instead of just 2 week chunk)
# 2. Add indicator variable for "is_collection"
# 3. Save data in multiple files to avoid out of memory errors

## Draft Visualizer

In [215]:
def get_card_distances(collection_list, cur_network, is_siamese=True):
    """ Get card distances for current network. Used for visualization """
    
    # Cardnames - for validation. 
    cardnames = val_dataloader.dataset.cardnames
    
    # Get collection vector
    collection_vector = torch.zeros([1, 271])
    cnt = Counter(collection_list)
    for card in cnt:
        
        # Validate cardname. 
        if card not in cardnames:
            raise Exception(f"{card} not in set. Please correct cardname.")

        # Add to collection vector. 
        card_index = cardnames.index(card)
        collection_vector[0, card_index] = cnt[card]
        
    # Get card and collection embeddings (support siamese & MLP)
    cur_network.eval()
    with torch.no_grad():
        if is_siamese:
            collection_embedding = cur_network(collection_vector)    
        else:
            # MLP equivalent. 
            card_distances = cur_network(collection_vector , torch.ones(271))
    if is_siamese:        
        card_embeddings = get_card_embeddings(cur_network)
        card_distances = get_distance(card_embeddings, collection_embedding)
    return card_distances

def get_percentile(card_distances, is_siamese=False, top_score=150):
    # TODO: omit basic lands. 
    card_distances = card_distances.reshape(-1) # Ensure correct shape. 
    min_distance = min(card_distances).item()
    max_distance = max(card_distances).item()
    if is_siamese:
        percentiles = [top_score * (max_distance - cd) / max_distance for cd in card_distances.tolist()]
    else:
        percentiles = [top_score * (cd - min_distance) / (max_distance - min_distance) for cd in card_distances.tolist()]
    return [round(p, 1) for p in percentiles]

# Example. 
# x = get_card_distances([c for c in collection["name"]], cur_network, is_siamese=True)
x = get_card_distances([c for c in collection["name"]], mlp_network, is_siamese=False)
p = get_percentile(x)

# pd.Series(p).describe()

In [216]:
# UI elements. 
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

# State variables for filters
rarity_options = ["All", "common", "uncommon", "common+uncommon", "rare", "mythic"]
color_options = ["All", "W", "G", "U", "R", "B", "Multicolor", "Colorless"]

rarity_filter = widgets.Dropdown(
    options=rarity_options,
    value="All",
    description="Rarity:",
)

color_filter = widgets.Dropdown(
    options=color_options,
    value="All",
    description="Color:",
)

def update_table():
    """Re-render the pick table and collection."""
    clear_output(wait=True)
    display_tables()

def make_pick(card):
    """Add card to the collection and update tables."""
    global collection
    collection = pd.concat([collection, pd.DataFrame([card])], ignore_index=True)
    update_table()

def remove_card(index):
    """Remove a card from the collection by index and update tables."""
    global collection
    collection = collection.drop(index).reset_index(drop=True)
    update_table()
    
# Function to reset the collection
def reset_collection(change=None):
    """Reset the collection (clear all cards)."""
    global collection
    collection = pd.DataFrame(columns=pick_table.columns)  # Empty collection
    update_table()

def display_tables():
    """Display pick table and collection with interactive buttons."""
    global pick_table

    # Update distances pick table.  
    collection_list = [n for n in collection["name"]]
    # cur_distances = get_card_distances(collection_list, cur_network) # Siamese
    # percentiles = get_percentile(cur_distances) Siamese
    cur_distances = get_card_distances(collection_list, mlp_network, is_siamese=False) # MLP
    percentiles = get_percentile(cur_distances, is_siamese=False) # MLP
    pick_table["distance"] = percentiles # Use percentiles for now. 
    
    if "p1p1_distance" not in pick_table.columns:
        p1p1_distances = get_card_distances([], mlp_network, is_siamese=False)
        p1p1_percentiles = get_percentile(p1p1_distances, is_siamese=False)
        pick_table["p1p1_distance"] = p1p1_percentiles
        
    pick_table["synergy"] = (pick_table["distance"] - pick_table["p1p1_distance"]).round(1)
        
    
    # Hide distances in collection table. 
    collection["distance"] = [""] * len(collection)
    
    # Apply filters to the pick table
    filtered_table = pick_table.copy()

    # If the rarity filter is "All", exclude cards with "Basic" rarity
    if rarity_filter.value == "All":
        filtered_table = filtered_table[filtered_table['rarity'] != "basic"]
    elif rarity_filter.value == "common+uncommon":
        filtered_table = filtered_table[filtered_table['rarity'].isin(["common", "uncommon"])]
    else:
        filtered_table = filtered_table[filtered_table['rarity'] == rarity_filter.value]
    
    if color_filter.value != "All":
        filtered_table = filtered_table[filtered_table['color_identity'] == color_filter.value]

    # Sort the filtered pick table by distance (ascending order)
    filtered_table = filtered_table.sort_values(by="distance", ascending=False)
    
    # Add the "New Draft" button to reset the collection
    new_draft_button = widgets.Button(description="New Draft", button_style="warning")
    new_draft_button.on_click(reset_collection)
    display(new_draft_button)
    
    # Display the filters
    filter_box = widgets.HBox([rarity_filter, color_filter])
    display(filter_box)

    # Get the maximum length of card names to align them
    max_name_length = filtered_table['name'].apply(len).max()
    max_name_length = max(max_name_length, 12)  # Minimum width for the name column is 12

    # Formatting function to align columns and display as text
    def format_row(row):
        return f"{row['name']:<{max_name_length}} | {row['rarity']:<9} | {row['color_identity']:<12} | {row['synergy']:>+7}| {row['distance']:>6}"

    # Display the filtered pick table with buttons
    print(f'{" Card Name":<{max_name_length}} | {"Rarity":<9} | {"Color":<12} | {"Synergy":>7}| {"Rating":>6}')
    for _, row in filtered_table.iterrows():
        row_widget = widgets.Output()
        with row_widget:
            print(format_row(row))

        pick_button = widgets.Button(description=f"Pick: {row['name']}", button_style="success")
        pick_button.on_click(lambda btn, card=row: make_pick(card.to_dict()))

        display(widgets.HBox([row_widget, pick_button]))

    # Display the collection with remove buttons (same format as pick table)
    print("\nCollection:")
    if not collection.empty:
        collection_widget = widgets.Output()
        with collection_widget:
            # Use the same format_row for collection as for pick table
            for _, row in collection.iterrows():
                print(format_row(row))

        remove_buttons = []
        for idx, row in collection.iterrows():
            remove_button = widgets.Button(description=f"Remove: {row['name']}", button_style="danger")
            remove_button.on_click(lambda btn, index=idx: remove_card(index))

            # Align text and remove button together in the same layout
            row_widget = widgets.Output()
            with row_widget:
                print(format_row(row))
            
            remove_buttons.append(widgets.HBox([row_widget, remove_button]))

        remove_buttons_box = widgets.VBox(remove_buttons)
        display(remove_buttons_box)
    else:
        print("Collection is empty.")

# Add observers to filters to trigger table updates
rarity_filter.observe(lambda change: update_table(), names='value')
color_filter.observe(lambda change: update_table(), names='value')

# Initial display
# display_tables()

In [217]:
# Get card data for visualization. 
pick_table = pd.read_csv("../data/BLB/cards.csv")
pick_table = pick_table[pick_table["expansion"].isin(["BLB", "SPG"])]
pick_table = pick_table[pick_table["name"].isin(val_dataloader.dataset.cardnames)]
pick_table = pick_table.groupby('name').first()
pick_table = pick_table.sort_values(by=["name"]).reset_index()
pick_table = pick_table[["name", "rarity", "color_identity"]]

# Fix color identity
pick_table["color_identity"] = pick_table["color_identity"].fillna("Colorless")
pick_table["color_identity"] = pick_table["color_identity"].apply(lambda x: "Multicolor" if (len(x) > 1 and x!="Colorless") else x)

# Dummy distance. 
pick_table["distance"] = [1] * len(pick_table)

# Initialize an empty collection DataFrame
collection = pd.DataFrame(columns=pick_table.columns)

In [218]:
# Initial display
display_tables()

Button(button_style='warning', description='New Draft', style=ButtonStyle())

 Card Name                 | Rarity    | Color        | Synergy| Rating



Collection:
